In [25]:
import os
import requests
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

Importando os dados

In [2]:
diretorio_principal = os.getcwd()
subdiretorio_dados = os.path.join(diretorio_principal, "Dados")

# Criar o diretório se não existir
os.makedirs(subdiretorio_dados, exist_ok=True)

In [3]:
anos = range(2008, 2024)
formato = "csv"
url_base = "http://www.camara.leg.br/cotas/"

for ano in anos:
    download = requests.get(url_base + f"Ano-{ano}.{formato}.zip")
    caminho_zip = os.path.join(subdiretorio_dados, f"Ano-{ano}.{formato}.zip")

    with open(caminho_zip, "wb") as file:
        file.write(download.content)

    # Extrair o conteúdo do arquivo ZIP
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(subdiretorio_dados)

    # # Remover o arquivo ZIP após extração (opcional)
    # os.remove(caminho_zip)

In [4]:
# Voltar ao diretório principal
os.chdir(diretorio_principal)

# Criar um DataFrame com os dados extraídos
despesas_totais = []

for arquivo_csv in os.listdir(subdiretorio_dados):
    if arquivo_csv.endswith('.csv') and not arquivo_csv.startswith("."):
        caminho_csv = os.path.join(subdiretorio_dados, arquivo_csv)
        despesas = pd.read_csv(caminho_csv, sep=";", encoding='UTF-8', low_memory=False)
        despesas_totais.append(despesas)

# Concatenar todos os DataFrames em um único DataFrame
despesas_completo = pd.concat(despesas_totais, ignore_index=True)

In [5]:
despesas_completo

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LIDERANÇA DO PDT,NaN,NaN,NaN,2011,NaN,NaN,54,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1042075,NaN,NaN,NaN,2715,5159141,NaN
1,LIDERANÇA DO PDT,NaN,NaN,NaN,2011,NaN,NaN,54,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1043845,NaN,NaN,NaN,2715,5165750,NaN
2,LIDERANÇA DO PDT,NaN,NaN,NaN,2011,NaN,NaN,54,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1050979,NaN,NaN,NaN,2715,5177451,NaN
3,LIDERANÇA DO PDT,NaN,NaN,NaN,2011,NaN,NaN,54,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1059955,NaN,NaN,NaN,2715,5227489,NaN
4,LIDERANÇA DO PDT,NaN,NaN,NaN,2011,NaN,NaN,54,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1069465,NaN,NaN,NaN,2715,5244951,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666836,Eliza Virgínia,7.972693e+10,220008.0,610.0,2019,PB,PP,56,998,PASSAGEM AÉREA - SIGEPA,...,0,ELIZA V DE SOUZA FERNANDES,JPA/BSB,0,0.0,NaN,NaN,3501,111862,NaN
4666837,Eliza Virgínia,7.972693e+10,220008.0,610.0,2019,PB,PP,56,998,PASSAGEM AÉREA - SIGEPA,...,0,ELIZA V DE SOUZA FERNANDES,REC/BSB,0,0.0,NaN,NaN,3501,113400,NaN
4666838,Eliza Virgínia,7.972693e+10,220008.0,610.0,2019,PB,PP,56,998,PASSAGEM AÉREA - SIGEPA,...,0,ELIZA V DE SOUZA FERNANDES,BSB/REC,0,0.0,NaN,NaN,3501,113848,NaN
4666839,Eliza Virgínia,7.972693e+10,220008.0,610.0,2019,PB,PP,56,998,PASSAGEM AÉREA - SIGEPA,...,0,ELIZA V DE SOUZA FERNANDES,JPA/BSB,0,0.0,NaN,NaN,3501,115984,NaN


Selecionando Colunas de Interesse

In [7]:
dataset = despesas_completo.loc[:,['sgUF', 'sgPartido', 'txtDescricao', 'txtPassageiro', 'txtTrecho', 'datPagamentoRestituicao', 'vlrRestituicao']]

In [8]:
dataset

,sgUF,sgPartido,txtDescricao,txtPassageiro,txtTrecho,datPagamentoRestituicao,vlrRestituicao
0,NaN,NaN,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,NaN,NaN,NaN
1,NaN,NaN,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,NaN,NaN,NaN
2,NaN,NaN,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,NaN,NaN,NaN
3,NaN,NaN,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,NaN,NaN,NaN
4,NaN,NaN,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4666836,PB,PP,PASSAGEM AÉREA - SIGEPA,ELIZA V DE SOUZA FERNANDES,JPA/BSB,NaN,NaN
4666837,PB,PP,PASSAGEM AÉREA - SIGEPA,ELIZA V DE SOUZA FERNANDES,REC/BSB,NaN,NaN
4666838,PB,PP,PASSAGEM AÉREA - SIGEPA,ELIZA V DE SOUZA FERNANDES,BSB/REC,NaN,NaN
4666839,PB,PP,PASSAGEM AÉREA - SIGEPA,ELIZA V DE SOUZA FERNANDES,JPA/BSB,NaN,NaN


Conferindo dados Selecionados

In [20]:
dataset.head(10)

,sgUF,sgPartido,txtDescricao,txtPassageiro,txtTrecho,datPagamentoRestituicao,vlrRestituicao
2820839,MT,DEM,PASSAGEM AÉREA - REEMBOLSO,BRUNO EDGAR SANTULLO;,Brasilia - Belo Horizonte,2017-03-01,538.69


In [21]:
dataset.tail(10)

,sgUF,sgPartido,txtDescricao,txtPassageiro,txtTrecho,datPagamentoRestituicao,vlrRestituicao
2820839,MT,DEM,PASSAGEM AÉREA - REEMBOLSO,BRUNO EDGAR SANTULLO;,Brasilia - Belo Horizonte,2017-03-01,538.69


In [11]:
dataset.shape

(4666841, 7)

In [22]:
dataset.describe()

,datPagamentoRestituicao,vlrRestituicao
count,1,1.00
mean,2017-03-01 00:00:00,538.69
min,2017-03-01 00:00:00,538.69
25%,2017-03-01 00:00:00,538.69
50%,2017-03-01 00:00:00,538.69
75%,2017-03-01 00:00:00,538.69
max,2017-03-01 00:00:00,538.69
std,NaN,NaN


In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 2820839 to 2820839
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sgUF                     1 non-null      object        
 1   sgPartido                1 non-null      object        
 2   txtDescricao             1 non-null      object        
 3   txtPassageiro            1 non-null      object        
 4   txtTrecho                1 non-null      object        
 5   datPagamentoRestituicao  1 non-null      datetime64[ns]
 6   vlrRestituicao           1 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 64.0+ bytes


Alterar tipos de dados

In [14]:
dataset['datPagamentoRestituicao'] = pd.to_datetime(dataset['datPagamentoRestituicao'], errors='coerce')

Limpeza de dados

In [16]:
#Remover duplicatas
dataset = dataset.drop_duplicates(keep="first")

In [ ]:
#Remover valores
#dataset = dataset.loc[dataset['coluna'] == 'valor']

In [ ]:
#Remover colunas desnecessárias
#dataset = dataset.drop(columns=['coluna1', 'coluna2'])

Dados faltantes

In [17]:
#Deletar linhas com dados faltantes
dataset = dataset.dropna()

In [ ]:
#Imputar valores
""" imputer = SimpleImputer(minssing_valeues = np.nan, strategy='mean')
imputer.fit(dataset[:,:])
imputer.transform(dataset[:, :]) """

In [ ]:
#Substituir valores
# dataset.replace(old_value, new_value, inplace=True)

Outliers

In [ ]:
# Diagrama de Tukey (boxplot)
gridsize_boxplot = (1, 2)

fig_boxplot = plt.figure(figsize=(12, 8))

boxplot1 = plt.subplot2grid(gridsize_boxplot, (0, 0))
boxplot2 = plt.subplot2grid(gridsize_boxplot, (0, 1))

boxplot1.set_title('Title 1', fontsize=8)
boxplot1.boxplot(dataset['col1'])

boxplot2.set_title('Title 2', fontsize=8)
boxplot2.boxplot(dataset['col2'])

In [ ]:
# Gráfico de dispersão
gridsize_scatter = (1, 2)

fig_scatter = plt.figure(figsize=(12, 8))

scatter1 = plt.subplot2grid(gridsize_scatter, (0, 0))
scatter2 = plt.subplot2grid(gridsize_scatter, (0, 1))

scatter1.set_title('Title 1', fontsize=8)
scatter1.scatter(dataset['col1'])

scatter2.set_title('Title 2', fontsize=8)
scatter2.scatter(dataset['col2'])

In [ ]:
# Histograma
gridsize_hist = (1, 2)

fig_hist = plt.figure(figsize=(12, 8))

hist1 = plt.subplot2grid(gridsize_hist, (0, 0))
hist2 = plt.subplot2grid(gridsize_hist, (0, 1))

hist1.set_title('Title 1', fontsize=8)
hist1.hist(dataset['col1'])

hist2.set_title('Title 2', fontsize=8)
hist2.hist(dataset['col2'])

In [ ]:
# Remover outliers - Winsorization

cols = list(dataset.columns)
cols.remove('col')

winsorization_flooring = 0.05
winsorization_capping = 1 - winsorization_flooring

not_selected_data_flooring = dataset[cols] <= dataset[cols].quantile(winsorization_flooring)
not_selected_data_capping = dataset[cols] >= dataset[cols].quantile(winsorization_capping)

not_selected_data_all = (not_selected_data_flooring & not_selected_data_capping)

data_index = dataset[cols][not_selected_data_all].index
dataset[cols].drop(data_index, inplace=True

Colinearidade

In [ ]:
# Scatterplots
sns.pairplot(X)

In [ ]:
# Pearson correlation coefficient
# The range for a correlation coefficient is between -1 and 1
# Typically, we say that two variables are correlated if 
# their correlation coefficient is greater than 0.5, or less than -0.5.
corr = X.corr()
print("Correlation Coefficients")
print(corr)

Por txtDescricao = PASSAGEM AÉREA 